# Segmenting and Clustering Neighborhoods in Toronto

## 1. Importing the required libraries

In [295]:
import numpy as np 
import pandas as pd
import folium 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import urllib.request
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

## 2. Web Scraping to read the List of postal codes in canada

In [296]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [297]:
opener = urllib.request.FancyURLopener({})
f = opener.open(url)

C:\Users\shima\Ana\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


In [298]:
# Reading the html file using pandas
data = pd.read_html(f)
data

[    Postcode           Borough  \
 0        M1A      Not assigned   
 1        M2A      Not assigned   
 2        M3A        North York   
 3        M4A        North York   
 4        M5A  Downtown Toronto   
 5        M6A        North York   
 6        M6A        North York   
 7        M7A  Downtown Toronto   
 8        M8A      Not assigned   
 9        M9A         Etobicoke   
 10       M1B       Scarborough   
 11       M1B       Scarborough   
 12       M2B      Not assigned   
 13       M3B        North York   
 14       M4B         East York   
 15       M4B         East York   
 16       M5B  Downtown Toronto   
 17       M5B  Downtown Toronto   
 18       M6B        North York   
 19       M7B      Not assigned   
 20       M8B      Not assigned   
 21       M9B         Etobicoke   
 22       M9B         Etobicoke   
 23       M9B         Etobicoke   
 24       M9B         Etobicoke   
 25       M9B         Etobicoke   
 26       M1C       Scarborough   
 27       M1C       

In [299]:
#Extracting the first element of the list
Toronto_DT = data[0]
Toronto_DT

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


## 3. Data Pre-processing

In [300]:
# Processing the cells that have an assigned borough and updating the Toronto_DT
Toronto_DT= Toronto_DT[Toronto_DT['Borough'] != 'Not assigned'].reset_index()

Toronto_DT.drop('index', axis=1, inplace=True)

#combining neighborhoods with the same post code into one row separated with a comma 
DF = Toronto_DT.groupby('Postcode').agg(lambda x: ','.join(x))
DF = DF.reset_index()

DF['Borough'] = DF['Borough'].str.split(',')

In [301]:
b_col = []
for i in range(0, 102):
    b_col.append(DF['Borough'][i][0])
    
b_col = pd.Series(b_col)
DF['Borough'] = b_col

In [302]:
# Checking missing values
print(DF['Borough'].isnull().sum())
print(DF['Neighbourhood'].isnull().sum())

1
0


In [303]:
DF['Borough'].replace(np.nan, 'Northwest', inplace= True)

In [304]:
DF.shape

(103, 3)

In [305]:
#Reading the cooardinate from the csv file
coordinate = pd.read_csv('Geospatial_Coordinates.csv')

In [306]:
coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 4. Adding the Latitude and Longitude

In [307]:
def append_LL(Post_code):
    for i in range(0,102):
        if DF.loc[i,'Postcode'] == coordinate.loc[i,'Postal Code']:
            DF['Latitude'] = coordinate['Latitude']
            DF['Longitude'] = coordinate['Longitude']

In [308]:
DF.apply(lambda x: append_LL(x['Postcode']), axis=1)

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
33     None
34     None
35     None
36     None
37     None
38     None
39     None
40     None
41     None
42     None
43     None
44     None
45     None
46     None
47     None
48     None
49     None
50     None
51     None
52     None
53     None
54     None
55     None
56     None
57     None
58     None
59     None
60     None
61     None
62     None
63     None
64     None
65     None
66     None
67     None
68     None
69     None
70     None
71     None
72     None
73     None
74     None
75     None
76     None
77     None
78     None
79     None
80     None
81     None
82     None
83  

In [309]:
DF.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 5. Creating maps

In [310]:
Lat, Lon = DF.loc[0, 'Latitude'], DF.loc[0,'Longitude']
Toronto_map = folium.Map(location= [Lat, Lon], zoom_start=10)

In [311]:
for lat, lng, borough, neighborhood in zip(DF['Latitude'],DF['Longitude'], DF['Borough'], DF['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#7997B4',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
Toronto_map

In [312]:
DF['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Northwest            1
Name: Borough, dtype: int64

In [313]:
East_Toronto = DF[DF['Borough'] == 'East Toronto'].reset_index().drop('index', axis = 1)
East_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [314]:
Lat, Lon = East_Toronto.loc[0, 'Latitude'], East_Toronto.loc[0,'Longitude']
East_Toronto_map = folium.Map(location= [Lat, Lon], zoom_start=12)
for lat, lng, borough, neighborhood in zip(East_Toronto['Latitude'],East_Toronto['Longitude'], East_Toronto['Borough'], East_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#7997B4',
        fill_opacity=0.7,
        parse_html=False).add_to(East_Toronto_map)  
East_Toronto_map

In [315]:
CLIENT_ID = '3NXVMIQ21F3VMFWEFC1I015JJGHY4CSSCVKGSODRCT3BUBMQ' # your Foursquare ID
CLIENT_SECRET = 'GWAEHFM1Z2JJU4YLWFKVHGOEEGJFGVJNMRAKGNDJ5TYX2CMV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3NXVMIQ21F3VMFWEFC1I015JJGHY4CSSCVKGSODRCT3BUBMQ
CLIENT_SECRET:GWAEHFM1Z2JJU4YLWFKVHGOEEGJFGVJNMRAKGNDJ5TYX2CMV


In [316]:
East_Toronto.loc[3,'Neighbourhood']

'Studio District'

In [317]:
st_dist = East_Toronto.loc[3,'Neighbourhood']
st_dist_lat = East_Toronto.loc[3,'Latitude']
st_dist_lon = East_Toronto.loc[3, 'Longitude']

print('Latitude and longitude values of {} are {}, {}.'.format(st_dist, st_dist_lat, st_dist_lon))

Latitude and longitude values of Studio District are 43.6595255, -79.340923.


In [318]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    st_dist_lat, 
    st_dist_lon, 
    500, 
    100)

In [319]:
json_data = requests.get(url).json()
json_data

{'meta': {'code': 200, 'requestId': '5e663c225fb726001bbb0a19'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Leslieville',
  'headerFullLocation': 'Leslieville, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.6640255045, 'lng': -79.33471445573701},
   'sw': {'lat': 43.6550254955, 'lng': -79.347131544263}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad7e958f964a520001021e3',
       'name': "Ed's Real Scoop",
       'location': {'address': '920 Queen St. E',
        'crossStreet': 'btwn Logan Ave. & Morse St.',
        'lat': 43.660655832455014,
        'lng': -79.3420187548006,
        'labeledLatLngs': 

In [320]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [321]:
venues = json_data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Ed's Real Scoop,Ice Cream Shop,43.660656,-79.342019
1,Leslieville Pumps,Sandwich Place,43.660892,-79.340626
2,Queen Books,Bookstore,43.660651,-79.342267
3,Te Aro,Coffee Shop,43.661373,-79.338577
4,The Bone House,Pet Store,43.660894,-79.341097
5,Hooked,Fish Market,43.660407,-79.343257
6,Purple Penguin Cafe,Café,43.660501,-79.342565
7,Mercury Espresso Bar,Coffee Shop,43.660806,-79.341241
8,Leslieville,Neighborhood,43.662070,-79.337856
9,WAYLABAR,Gay Bar,43.661234,-79.339597


In [322]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [323]:
Venues = getNearbyVenues(names=DF['Neighbourhood'],latitudes=DF['Latitude'],longitudes=DF['Longitude'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [324]:
nearby_venues.head()

,name,categories,lat,lng
0,Ed's Real Scoop,Ice Cream Shop,43.660656,-79.342019
1,Leslieville Pumps,Sandwich Place,43.660892,-79.340626
2,Queen Books,Bookstore,43.660651,-79.342267
3,Te Aro,Coffee Shop,43.661373,-79.338577
4,The Bone House,Pet Store,43.660894,-79.341097


In [325]:
nearby_venues.shape

(42, 4)

In [326]:
Venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [327]:
Venues.shape

(2222, 7)

In [328]:
Venues.groupby('Neighborhood').first()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
Agincourt,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",43.815252,-79.284577,Port Royal Park,43.815477,-79.289773,Park
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",43.739416,-79.588437,Subway,43.742645,-79.589643,Sandwich Place
"Alderwood,Long Branch",43.602414,-79.543484,Il Paesano Pizzeria & Restaurant,43.601280,-79.545028,Pizza Place
"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259,Wolfie's Deli,43.754875,-79.442438,Deli / Bodega
Bayview Village,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant
"Bedford Park,Lawrence Manor East",43.733283,-79.419750,Aroma Espresso Bar,43.735975,-79.420391,Café
Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store


In [329]:
Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [330]:
Venues_Analyzed = pd.get_dummies(Venues[['Venue Category']]) 
Venues_DF = pd.concat([Venues,Venues_Analyzed], axis=1)
Venues_DF= Venues_DF.drop('Venue Category', axis=1)
Venues_DF.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,Venue Category_Art Gallery,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Garage,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Baseball Stadium,Venue Category_Basketball Court,Venue Category_Basketball Stadium,Venue Category_Beach,Venue Category_Bed & Breakfast,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Belgian Restaurant,Venue Category_Bike Shop,Venue Category_Bistro,Venue Category_Board Shop,Venue Category_Boat or Ferry,Venue Category_Bookstore,Venue Category_Boutique,Venue Category_Brazilian Restaurant,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bridal Shop,Venue Category_Bubble Tea Shop,Venue Category_Building,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Bus Stop,Venue Category_Business Service,Venue Category_Butcher,Venue Category_Cafeteria,Venue Category_Café,Venue Category_Cajun / Creole Restaurant,Venue Category_Camera Store,Venue Category_Candy Store,Venue Category_Caribbean Restaurant,Venue Category_Cheese Shop,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Church,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_College Arts Building,Venue Category_College Auditorium,Venue Category_College Cafeteria,Venue Category_College Gym,Venue Category_College Rec Center,Venue Category_College Stadium,Venue Category_Colombian Restaurant,Venue Category_Comfort Food Restaurant,Venue Category_Comic Shop,Venue Category_Concert Hall,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Coworking Space,Venue Category_Creperie,Venue Category_Cuban Restaurant,Venue Category_Cupcake Shop,Venue Category_Curling Ice,Venue Category_Dance Studio,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Distribution Center,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Empanada Restaurant,Venue Category_Ethiopian Restaurant,Venue Category_Event Space,Venue Category_Falafel Restaurant,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Field,Venue Category_Filipino Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Fish Market,Venue Category_Flea Market,Venue Category_Flower Shop,Venue Category_Food,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_Food Truck,Venue Category_Fountain,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Fruit & Vegetable Store,Venue Category_Furniture / Home Store,Venue Category_Gaming Cafe,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gas Station,Venue Category_Gastropub,Venue Category_Gay Bar,Venue Category_General Entertainment,Venue Category_General Travel,Venue Category_German Restaurant,Venue Category_Gift Shop,Venue Category_Gluten-free Restaurant,Venue Category_Golf Course,Venue Category_Gourmet Shop,Ve

In [331]:
Venues_Based_Neig= Venues_DF.groupby('Neighborhood').mean().reset_index()

In [332]:
Venues_Based_Neig.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,Venue Category_Art Gallery,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Garage,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Baseball Stadium,Venue Category_Basketball Court,Venue Category_Basketball Stadium,Venue Category_Beach,Venue Category_Bed & Breakfast,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Belgian Restaurant,Venue Category_Bike Shop,Venue Category_Bistro,Venue Category_Board Shop,Venue Category_Boat or Ferry,Venue Category_Bookstore,Venue Category_Boutique,Venue Category_Brazilian Restaurant,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bridal Shop,Venue Category_Bubble Tea Shop,Venue Category_Building,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Bus Stop,Venue Category_Business Service,Venue Category_Butcher,Venue Category_Cafeteria,Venue Category_Café,Venue Category_Cajun / Creole Restaurant,Venue Category_Camera Store,Venue Category_Candy Store,Venue Category_Caribbean Restaurant,Venue Category_Cheese Shop,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Church,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_College Arts Building,Venue Category_College Auditorium,Venue Category_College Cafeteria,Venue Category_College Gym,Venue Category_College Rec Center,Venue Category_College Stadium,Venue Category_Colombian Restaurant,Venue Category_Comfort Food Restaurant,Venue Category_Comic Shop,Venue Category_Concert Hall,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Coworking Space,Venue Category_Creperie,Venue Category_Cuban Restaurant,Venue Category_Cupcake Shop,Venue Category_Curling Ice,Venue Category_Dance Studio,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Distribution Center,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Empanada Restaurant,Venue Category_Ethiopian Restaurant,Venue Category_Event Space,Venue Category_Falafel Restaurant,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Field,Venue Category_Filipino Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Fish Market,Venue Category_Flea Market,Venue Category_Flower Shop,Venue Category_Food,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_Food Truck,Venue Category_Fountain,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Fruit & Vegetable Store,Venue Category_Furniture / Home Store,Venue Category_Gaming Cafe,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gas Station,Venue Category_Gastropub,Venue Category_Gay Bar,Venue Category_General Entertainment,Venue Category_General Travel,Venue Category_German Restaurant,Venue Category_Gift Shop,Venue Category_Gluten-free Restaurant,Venue Category_Golf Course,Venue Category_Gourmet Shop,Venue Ca

In [333]:
Venues_Based_Neig.shape

(100, 276)

In [334]:
top_venues = 5
for hood in Venues_Based_Neig['Neighborhood']:
    print("________________________"+hood+"__________________________")
    temp = Venues_Based_Neig[Venues_Based_Neig['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','frequency']
    temp = temp.iloc[1:]
    temp['frequency'] = temp['frequency'].astype(float)
    temp = temp.round({'frequency': 2})
    print(temp.sort_values('frequency', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

________________________Adelaide,King,Richmond__________________________
                            venue  frequency
0           Neighborhood Latitude      43.65
1                  Venue Latitude      43.65
2      Venue Category_Coffee Shop       0.07
3       Venue Category_Restaurant       0.05
4  Venue Category_Thai Restaurant       0.04


________________________Agincourt__________________________
                                      venue  frequency
0                     Neighborhood Latitude      43.79
1                            Venue Latitude      43.79
2             Venue Category_Breakfast Spot       0.25
3  Venue Category_Latin American Restaurant       0.25
4               Venue Category_Skating Rink       0.25


________________________Agincourt North,L'Amoreaux East,Milliken,Steeles East__________________________
                        venue  frequency
0       Neighborhood Latitude      43.82
1              Venue Latitude      43.82
2   Venue Category_Playground       

4           Venue Category_Restaurant       0.03


________________________Clairlea,Golden Mile,Oakridge__________________________
                          venue  frequency
0         Neighborhood Latitude      43.71
1                Venue Latitude      43.71
2         Venue Category_Bakery       0.22
3       Venue Category_Bus Line       0.22
4  Venue Category_Metro Station       0.11


________________________Clarks Corners,Sullivan,Tam O'Shanter__________________________
                                 venue  frequency
0                Neighborhood Latitude      43.78
1                       Venue Latitude      43.78
2           Venue Category_Pizza Place       0.18
3  Venue Category_Fast Food Restaurant       0.09
4          Venue Category_Noodle House       0.09


________________________Cliffcrest,Cliffside,Scarborough Village West__________________________
                                venue  frequency
0               Neighborhood Latitude      43.72
1                      Ve

4            Venue Category_Drugstore       0.33


________________________Northwood Park,York University__________________________
                                   venue  frequency
0                  Neighborhood Latitude      43.77
1                         Venue Latitude      43.76
2  Venue Category_Furniture / Home Store       0.29
3    Venue Category_Caribbean Restaurant       0.14
4          Venue Category_Massage Studio       0.14


________________________Parkdale,Roncesvalles__________________________
                             venue  frequency
0            Neighborhood Latitude      43.65
1                   Venue Latitude      43.65
2    Venue Category_Breakfast Spot       0.14
3         Venue Category_Gift Shop       0.14
4  Venue Category_Cuban Restaurant       0.07


________________________Parkwoods__________________________
                              venue  frequency
0             Neighborhood Latitude      43.75
1                    Venue Latitude      43.75
2  

In [335]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [336]:

T_clustering = Venues_Based_Neig.drop('Neighborhood', axis=1)

kmeans = KMeans(n_clusters= 5 , random_state=0).fit(T_clustering)

kmeans.labels_

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 3, 4, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 3, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3])

In [337]:
Venues_Based_Neig['cluster'] = kmeans.labels_

In [339]:
latitude, longitude = DF.loc[0, 'Latitude'], DF.loc[0,'Longitude']
Tr_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, poi, cluster in zip(Venues_Based_Neig['Neighborhood Latitude'], Venues_Based_Neig['Neighborhood Longitude'], Venues_Based_Neig['Neighborhood'], Venues_Based_Neig['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Tr_clusters)
       
Tr_clusters